In [28]:
import tensorflow as tf

data_x = [[0.,0.,1.],[1.,1.,1.],[1.,2.,1.],[2.,1.,1.],[2.,2.,1.],[2.,0.,1.],[0.,2.,1.],[3.,3.,1.],[0.,3.,1.],[3.,0.,1.],[4.,4.,1.],[0.,4.,1,],[4.,0.,1.]]
data_y = [[1.],[6.],[9.],[8.],[11.],[5.],[7.],[16.],[10.],[7.],[21.],[13.],[9.]]
batch_size = len(data_x)

# Input and Output. No batch_size for simplicity.
x = tf.placeholder(shape=[batch_size,3], dtype=tf.float32, name="x")
y = tf.placeholder(shape=[batch_size,1], dtype=tf.float32, name="y")

# Weight and bias.
#W = tf.Variable(tf.ones(shape=[1]), dtype=tf.float32, name="W")
#b = tf.Variable(tf.zeros(shape=[1]), dtype=tf.float32, name="b")
wrt_variables = tf.Variable(tf.ones(shape=[3,1]), dtype=tf.float32, name="wrt")
z = tf.Variable(tf.ones(shape=[3,1]), dtype=tf.float32, name="z")
c = tf.constant
# Making a prediction and comparing it to the true output
wrt_variables_list = tf.unstack(wrt_variables);
#pred = x * wrt_variables_list[0] + wrt_variables_list[1]
pred = tf.matmul(x,wrt_variables_list)
loss = 0.5 * (y - pred)**2/batch_size
#loss = tf.reduce_mean(0.5 * (y - pred)**2)
loss = tf.unstack(loss)

[Hess23,grad]=Hess_Comp(loss,wrt_variables_list);
Jacob = Jaco_comp(loss,wrt_variables_list)
Hess=tf.matmul(tf.transpose(Jacob),tf.stack(Jacob)); 


cg_wrt_variables = z;

for iteration in range(5):
    #cg_loss = (grad - tf.matmul(Hess,cg_wrt_variables))**2;
    cg_wrt_variables_list = tf.unstack(cg_wrt_variables)
    cg_loss = (grad - tf.matmul(Hess,cg_wrt_variables_list))**2
    [H,G] = Hess_Comp(cg_loss,cg_wrt_variables_list);
    d=-G;
    for index in range (0,len(cg_wrt_variables_list)-1):
#    for index in range (0,1):
        Alpha_N= tf.matmul(-tf.transpose(G),d);
        Alpha_D= tf.matmul(tf.matmul(tf.transpose(d),H),d);
        Alpha = tf.divide(Alpha_N,Alpha_D);
        cg_update_directions = Alpha*d    

        cg_wrt_variables = tf.assign_add(cg_wrt_variables,cg_update_directions)
        
        cg_wrt_variables_list = tf.unstack(cg_wrt_variables);
        cg_loss = (grad - tf.matmul(Hess,cg_wrt_variables_list))**2
        [H1,G]=Hess_Comp(cg_loss,cg_wrt_variables_list)
        Beta_N= tf.matmul(tf.matmul(tf.transpose(G),H),d);
        Beta_D= tf.matmul(tf.matmul(tf.transpose(d),H),d);
        Beta = tf.divide(Beta_N,Beta_D);
        Beta_update_directions = tf.unstack(Beta*d)
        d=tf.subtract(Beta_update_directions,G)
        [H,G]=Hess_Comp(cg_loss,cg_wrt_variables_list)
update_directions = -cg_wrt_variables;
update_directions=(tf.unstack(update_directions))
print(update_directions)

wrt_variables = tf.assign_add(wrt_variables,update_directions)



[<tf.Tensor 'unstack_146:0' shape=(1,) dtype=float32>, <tf.Tensor 'unstack_146:1' shape=(1,) dtype=float32>, <tf.Tensor 'unstack_146:2' shape=(1,) dtype=float32>]


In [1]:
def Jaco_comp(y,x_list):
    Jaco=[];Jaco1=[];
    Jaco.append([tf.gradients(y_, x_list) for y_ in y])
    for i in range(0,len(Jaco[0])):
        for j in range(0,len(Jaco[0][i])):
            if Jaco[0][i][j] == None:
                Jaco1.append(cons(0))   
            else: 
                Jaco1.append(Jaco[0][i][j])
    Jaco1 = tf.reshape(Jaco1,[len(y),len(x_list)])
    return(Jaco1)

In [2]:
def Hess_Comp(y,x_list):
    grads=[];H=[];grads1=[];H1=[];
    grads.append(tf.gradients(y, x_list))
    for i in range(0,len(grads[0])):
        if grads[0][i] == None:
            grads1.append(cons(10))   
        else: 
            grads1.append(grads[0][i])
    H.append([tf.gradients(y_, x_list) for y_ in grads1])
    for i in range(0,len(H[0])):
        for j in range(0,len(H[0][i])):
            if H[0][i][j] == None:
                H1.append(cons(10))   
            else: 
                H1.append(H[0][i][j])
    H1 = tf.reshape(H1,[len(x_list),len(x_list)])
    grads1 = tf.reshape(grads1,[len(x_list),1])

    return(H1,grads1)

In [3]:
def cons(x):
    return tf.constant(x, dtype=tf.float32)

In [30]:


sess = tf.Session()
print("Hi")

sess.run(tf.global_variables_initializer())

initial_loss = sess.run(
    loss,
    feed_dict={
        x: data_x,
        y: data_y
    }
)
print("Initial loss:", initial_loss)    

for iteration in range(20):
    g1,g = sess.run(
        [wrt_variables,Hess],
        feed_dict={
            x: data_x,
            y: data_y
        }
    )
    comp_loss = sess.run(
        loss,
        feed_dict={
            x: data_x,
            y: data_y
        }
    )
    #print(h)
    print("Loss after iteration {}: {}".format(iteration+1, sum(comp_loss)/len(data_x)))
    if(sum(comp_loss)/len(data_x)<0.15):
        break
        
#prediction = data_x *g1[0] +g1[1];
print("Wrt_var:",g1)

    
#logs_path = "C:/Users/suraj/Logs"
#writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
#print("Prediction:", sess.run(pred, feed_dict={x: data_x}))
#print("Expected:", data_y)
sess.close();

Hi
Initial loss: [array([ 0.], dtype=float32), array([ 0.34615386], dtype=float32), array([ 0.96153843], dtype=float32), array([ 0.61538464], dtype=float32), array([ 1.38461542], dtype=float32), array([ 0.15384616], dtype=float32), array([ 0.61538464], dtype=float32), array([ 3.11538458], dtype=float32), array([ 1.38461542], dtype=float32), array([ 0.34615386], dtype=float32), array([ 5.53846169], dtype=float32), array([ 2.46153855], dtype=float32), array([ 0.61538464], dtype=float32)]
Loss after iteration 1: [ 0.93717158]
Loss after iteration 2: [ 0.52250922]
Loss after iteration 3: [ 0.31296828]
Loss after iteration 4: [ 0.39318922]
Loss after iteration 5: [ 0.10323144]
Wrt_var: [[ 2.28496599]
 [ 2.50266886]
 [-0.03239131]]


In [21]:
data_y

[[1.0], [6.0], [9.0], [8.0]]

In [24]:
data_x

[[0.0, 0.0, 1.0], [1.0, 1.0, 1.0], [1.0, 2.0, 1.0], [2.0, 1.0, 1.0]]